In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
df.columns[df.isnull().sum() > 0]

In [ ]:
df.fillna(method='bfill', inplace=True)
df.fillna(method='ffill', inplace=True)

In [ ]:
Numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
Categorical_columns = set(df.columns).difference(set(Numeric_columns))
print(Categorical_columns)

In [ ]:
see = df[Categorical_columns]
see.head()

In [ ]:
df['edjefe'] = df['edjefe'].replace({'no': 0, 'yes': 1}).astype(float)
df['edjefa'] = df['edjefa'].replace({'no': 0, 'yes': 1}).astype(float)

In [ ]:
df['dependency'] = np.sqrt(df['SQBdependency'])


In [ ]:
col_drops =  ['Id', 'idhogar']
df.drop(col_drops, axis= 1, inplace= True)

In [ ]:
Numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
Categorical_columns = set(df.columns).difference(set(Numeric_columns))

In [ ]:
train_set = df

In [ ]:
df['rent_per_adult'] = df['v2a1']/df['hogar_adul']
df['rent_per_person'] = df['v2a1']/df['hhsize']

df['overcrowding_room_and_bedroom'] = (df['hacdor'] + df['hacapo'])/2

df['no_appliances'] = df['refrig'] + df['computer'] + df['television']

df['r4h1_percent_in_male'] = df['r4h1'] / df['r4h3']
df['r4m1_percent_in_female'] = df['r4m1'] / df['r4m3']
df['r4h1_percent_in_total'] = df['r4h1'] / df['hhsize']
df['r4m1_percent_in_total'] = df['r4m1'] / df['hhsize']
df['r4t1_percent_in_total'] = df['r4t1'] / df['hhsize']

df['adult'] = df['hogar_adul'] - df['hogar_mayor']
df['dependency_count'] = train_set['hogar_nin'] + df['hogar_mayor']
df['dependency'] = df['dependency_count'] / df['adult']
df['child_percent'] = df['hogar_nin']/df['hogar_total']
df['elder_percent'] = df['hogar_mayor']/df['hogar_total']
df['adult_percent'] = df['hogar_adul']/df['hogar_total']

df['rent_per_bedroom'] = df['v2a1']/df['bedrooms']
df['adults_per_bedroom'] = df['adult']/df['bedrooms']
df['child_per_bedroom'] = df['hogar_nin']/df['bedrooms']
df['male_per_bedroom'] = df['r4h3']/df['bedrooms']

df['female_per_bedroom'] = df['r4m3']/df['bedrooms']
df['bedrooms_per_person_household'] = df['hhsize']/df['bedrooms']

df['tablet_per_person_household'] = df['v18q1']/df['hhsize']
df['phone_per_person_household'] = df['qmobilephone']/df['hhsize']

df['age_12_19'] = df['hogar_nin'] - df['r4t1']

df['rent_per_room'] = df['v2a1']/df['rooms']
df['bedroom_per_room'] = df['bedrooms']/df['rooms']
df['elder_per_room'] = df['hogar_mayor']/df['rooms']
df['adults_per_room'] = df['adult']/df['rooms']
df['child_per_room'] = df['hogar_nin']/df['rooms']
df['male_per_room'] = df['r4h3']/df['rooms']
df['female_per_room'] = df['r4m3']/df['rooms']
df['room_per_person_household'] = df['hhsize']/df['rooms']
df['escolari_age'] = df['escolari']/df['age']

df['rez_esc_escolari'] = df['rez_esc']/df['escolari']
df['rez_esc_r4t1'] = df['rez_esc']/df['r4t1']
df['rez_esc_r4t2'] = df['rez_esc']/df['r4t2']
df['rez_esc_r4t3'] = df['rez_esc']/df['r4t3']
df['rez_esc_age'] = df['rez_esc']/df['age']

In [ ]:
list_drop = list(df.columns[df.isna().any()].tolist())
print(list_drop)
df.drop(list_drop, axis= 1, inplace= True)

In [ ]:
for i in list(df.columns.to_series()[np.isinf(df).any()]):
    df[i].replace([np.inf, -np.inf], 0, inplace=True)
print(list(df.columns.to_series()[np.isinf(df).any()]))

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
x_1 = df.drop('Target', axis=1)
y = df['Target']

In [ ]:
x_train,x_test, y_train,y_test = train_test_split(x_1,y, random_state=42)

In [ ]:
Xg=XGBClassifier()
Lgbm=LGBMClassifier(n_estimators=100,learning_rate=0.1,random_state=42,num_leaves=200)
    
Cataboost=CatBoostClassifier(depth=int(9.253),n_estimators=int(514.1))
estimators = [('Cataboost', Cataboost), ('Xg', Xg), ('Lgbm', Lgbm)]

clf = StackingClassifier(estimators=estimators)

In [ ]:
clf.fit(x_train, y_train)